In [ ]:
import requests
import csv
import re
from bs4 import BeautifulSoup

In [ ]:
f = open("kovo-female.csv","w", encoding="utf-8-sig")
writer = csv.writer(f)

In [ ]:
head = "NO 현대건설 GS칼텍스 KGC인삼공사 한국도로공사 흥국생명 IBK기업은행 페퍼저축은행".split(" ")
writer.writerow(head)

In [ ]:
ratings = {}
k = 28
home_advantage = 22

In [ ]:
ratings['현대건설'] = 1500
ratings['GS칼텍스'] = 1500
ratings['KGC인삼공사'] = 1500
ratings['한국도로공사'] = 1500
ratings['흥국생명'] = 1500

In [ ]:
for season in range(1,19):
    season_home = 0
    season_away = 0
    if season == 8:
        ratings['IBK기업은행'] = 1500
    if season == 18:
        ratings['페퍼저축은행'] = 1500
    data = [' ']+[ratings[key] for key in ratings]
    writer.writerow(data)
    if season <10:
        season = "0"+str(season)
    for game_number in range(1,300):
        url = "https://www.kovo.co.kr/game/v-league/11141_game-summary.asp?season=0"+str(season)+"&g_part=201&r_round=1&g_num="+str(game_number)+"&"
        res = requests.get(url)
        print(url)
        soup = BeautifulSoup(res.text,"lxml")
        team_names = soup.find_all("span",attrs={"class":"team"}) #팀 이름을 리스트로 가져옴
        if team_names == []:
            for key in ratings:
                if ratings[key] != 0:
                    ratings[key] = ratings[key]+(1500-ratings[key])/2
            break
        match_date = soup.find("table", attrs={"class" : "lst_recentgame lst_result mt10"}).find("thead").find("tr").find("th").get_text()[1:21]
        home_team_name = team_names[0].get_text()
        away_team_name = team_names[1].get_text()
        if home_team_name == "도로공사":
            home_team_name = "한국도로공사"
        if away_team_name == "도로공사":
            away_team_name = "한국도로공사"
        if home_team_name == "KT&G":
            home_team_name = "KGC인삼공사"
        if away_team_name == "KT&G":
            away_team_name = "KGC인삼공사"
        if home_team_name == "인삼공사":
            home_team_name = "KGC인삼공사"
        if away_team_name == "인삼공사":
            away_team_name = "KGC인삼공사"
        if home_team_name not in ratings:
            continue
        if away_team_name not in ratings:
            continue
        results = soup.find_all("p",attrs={"class":re.compile("^result")}) #승패 결과를 리스트로 가져옴
        points = soup.find_all("p",attrs={"class":"num"}) #세트 스코어를 리스트로 가져옴
        if points == []:
            break
        if season == 18:
            if points[0].get_text() == points[1].get_text():
                break
        expected_score = 1/(1+10**((ratings[away_team_name]-ratings[home_team_name]-home_advantage)/400))
        if int(points[0].get_text()) > int(points[1].get_text()):
            total_home += 1
            season_home += 1
            ratings[home_team_name] = ratings[home_team_name]+k*(1-expected_score)
            ratings[away_team_name] = ratings[away_team_name]-k*(1-expected_score)
        else:
            ratings[home_team_name] = ratings[home_team_name]+k*(0-expected_score)
            ratings[away_team_name] = ratings[away_team_name]-k*(0-expected_score)
            total_away += 1
            season_away += 1
        data = [match_date]+[ratings[key] for key in ratings]
        writer.writerow(data)

In [ ]:
for season in range(19):
    data = [' ']+[ratings[key] for key in ratings]
    writer.writerow(data)